In [3]:
!cat ~/Developer/ctf/imaginaryctf/attatchments/univariate/chall.sage

from Crypto.Util.number import getPrime, bytes_to_long
from secret import flag

p = getPrime(512)
q = getPrime(512)
n = p*q

m = bytes_to_long(flag.encode())
e = 65537
c = pow(m,e,n)

P.<x> = PolynomialRing(ZZ)
x = P.gens()[0]

terms = [x**i for i in range(137)]

T = RealDistribution('gaussian', 2)
coefs = [round(T.get_random_element()) for _ in range(len(terms))]

f = sum([term*coef for term,coef in zip(terms,coefs)])
w = pow(2,f(p),n)

with open('out.txt', 'w') as file:
    file.write(f'{n = }\n')
    file.write(f'{e = }\n')
    file.write(f'{c = }\n')
    file.write(f'{f = }\n')
    file.write(f'{w = }\n')


In [27]:
from pathlib import Path
path = "Developer/ctf/imaginaryctf/attatchments/out.txt"

In [61]:
from sage.all import *
from Crypto.Util.number import long_to_bytes

# Read values from out.txt
with open(Path(path).absolute(), 'r') as file:
    lines = file.readlines()
    n = Integer(lines[0].split('=')[1].strip())
    e = Integer(lines[1].split('=')[1].strip())
    c = Integer(lines[2].split('=')[1].strip())
    f_str = lines[3].split('=')[1].strip()
    w = Integer(lines[4].split('=')[1].strip())

# Define the polynomial ring and parse f
P = PolynomialRing(ZZ, 'x')
x = P.gens()[0]
f = sage_eval(f_str, locals={'x': x})

f

-x^136 + x^135 - 2*x^134 - 4*x^132 + 2*x^130 - x^128 - 3*x^127 + 4*x^126 + 3*x^125 + 3*x^124 + x^123 + x^122 - 5*x^121 - 3*x^120 - x^119 - x^118 + x^117 + x^116 - 4*x^114 - 2*x^112 + 2*x^110 + x^109 + 2*x^108 - 2*x^107 + 3*x^106 - x^104 + 2*x^103 - x^102 + x^101 - 2*x^100 + 3*x^99 - 2*x^98 - x^97 - x^96 - 3*x^95 - x^94 - 2*x^93 - 2*x^91 + 3*x^90 - 2*x^89 - 2*x^88 + x^86 + x^85 - 2*x^84 - 3*x^83 + 2*x^82 + 3*x^79 - x^76 + 2*x^75 - x^74 + x^71 - 5*x^70 - x^67 + x^66 + x^65 + x^63 - x^61 + x^59 - 2*x^58 + 6*x^56 + x^55 + 3*x^54 - x^53 + 2*x^52 + 3*x^51 + x^50 + 2*x^49 + 3*x^47 + 2*x^46 - 4*x^45 + 3*x^44 + 3*x^43 - x^42 - 2*x^40 - 5*x^39 + x^38 + x^37 + 2*x^36 + 2*x^35 + x^34 - x^33 + x^32 - 5*x^31 + x^30 + x^29 + 2*x^28 - 2*x^27 + 3*x^26 - x^25 - x^23 - x^22 - 3*x^21 + 2*x^20 - x^19 - x^17 + 2*x^16 - 2*x^15 - 2*x^14 - 2*x^13 - 2*x^12 + 2*x^11 - 2*x^9 + 3*x^8 - 4*x^7 + 2*x^6 - 2*x^5 - 5*x^4 - 3*x^3 + 5*x^2 - 2

In [65]:
(w * pow(2,-f(n-1),n)) % n

39127159436311440488373342736700330343271349705164427413196450140415044808514209327289843535347229362191563463913194554859158732886761005597789150818501417821223878559525326766325432433770386567687753389293972089441560698611536681170015124510337513321718569381624570547740148983024217967691451534202069817990

# Testing

In [57]:
p,q = 13,7
N = p*q

W = pow(2,f(q),N)
W

64

In [60]:
(W * pow(2,f(N),N)) % N

1

In [22]:
# Step 1: Get the constant term c0
f0 = f(0) % n

# Step 2: Compute v = 2^{c0} mod n
v = pow(2, f0, n)

f0,v,w

(151510886600487624888537926759375027338192556324330182365859112926770109752858284462159488504727238764120612593911292154858008775463001345641311051184326218974685701057787672193003745574697137968457609530135969033403360561333863943223407215732526198691453110628598401583407984162075630768455052482583101773635,
 52307052132037551886204118371667599039556941941716906779010996828796128663244343794560675987889231817688456130899116466828559966793102740645699784700424572937840292820958998067279985095143898652048413517712918012031543374791724586964847768669968297437586215131325433547208206440717949898318630070997904547965,
 86258923706084556733053644452456806418792871483898871193707132372143291757396867798433017660985422614532352743658877188445517898648519256573663299464811234251773841741466280567326570167017786562044635756348763128567054349991798640926148221279889174229551074668002853442182664523748992260830782387602048836221)

In [ ]:
!cat 

In [50]:
# Step 3: Compute p = gcd(w - v, n)
p = gcd(w - v, n)

# Check if we got a proper factor
if p == 1 or p == n:
    print("Failed to factor n.")
else:
    # Step 4: Compute q
    q = n // p

    # Step 5: Compute phi(n)
    phi_n = (p - 1) * (q - 1)

    # Step 6: Compute d
    d = inverse_mod(e, phi_n)

    # Step 7: Decrypt m
    m = pow(c, d, n)

    # Step 8: Convert to flag
    flag = long_to_bytes(m)
    print("Flag:", flag.decode())

Failed to factor n.
